In [6]:
import pandas as pd
import numpy as np


In [7]:
df = pd.read_csv('merged_spoiler.csv')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1378032 entries, 0 to 1378031
Data columns (total 12 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   user_id                1378032 non-null  object 
 1   timestamp              1378032 non-null  object 
 2   review_sentences       1378032 non-null  object 
 3   rating                 1378032 non-null  int64  
 4   has_spoiler            1378032 non-null  bool   
 5   book_id                1378032 non-null  int64  
 6   review_id              1378032 non-null  object 
 7   genre                  1378032 non-null  object 
 8   description            1370987 non-null  object 
 9   publication_year       1241707 non-null  float64
 10  ratings_count          1378032 non-null  int64  
 11  review_sentences_join  1378032 non-null  object 
dtypes: bool(1), float64(1), int64(3), object(7)
memory usage: 117.0+ MB


In [9]:
drop_cols = ['user_id' , 'timestamp', 'book_id' , 'review_id', 'publication_year' ]
df= df.drop(drop_cols , axis =1)

In [10]:
df.isnull().sum()

review_sentences            0
rating                      0
has_spoiler                 0
genre                       0
description              7045
ratings_count               0
review_sentences_join       0
dtype: int64

In [11]:
df[df["description"].isnull()]

,review_sentences,rating,has_spoiler,genre,description,ratings_count,review_sentences_join
59,"[[0, 'I read this after I went fly fishing for...",5,False,fiction,NaN,4526,I read this after I went fly fishing for the f...
69,"[[0, 'This book was pure Gold - it was impossi...",5,False,other,NaN,16821,This book was pure Gold - it was impossible to...
288,"[[0, '5 Sexy and Edgy Stars!'], [0, '""Even the...",5,False,romance,NaN,1122,"5 Sexy and Edgy Stars!""Even the best of us isn..."
629,"[[0, ""I was thrilled to be able to read this s...",4,False,other,NaN,2469,I was thrilled to be able to read this super s...
1123,"[[0, ""I'm not really sure what I was expecting...",2,False,fiction,NaN,7350,I'm not really sure what I was expecting from ...
...,...,...,...,...,...,...,...
1377344,"[[0, 'An inspiring book.'], [0, 'Enough said :...",3,False,fiction,NaN,1342863,An inspiring book.Enough said :)SALYMAR'S BETT...
1377538,"[[0, ""A sequel that's better than the original...",3,False,fiction,NaN,13749,A sequel that's better than the original?Given...
1377539,"[[0, ""This sat on my 'to read' pile for six mo...",4,False,young-adult,NaN,6340,This sat on my 'to read' pile for six months a...
1377557,"[[0, 'How I feel about this book depends on ho...",3,False,fiction,NaN,55039,How I feel about this book depends on how I lo...


In [12]:
# Train test split
from sklearn.model_selection import train_test_split
X = df.drop(['has_spoiler'] , axis =1)
Y = df['has_spoiler']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42, stratify = Y )

In [8]:
del df

In [9]:
len(X_train)

1033524

In [10]:
y_train.value_counts()

False    966304
True      67220
Name: has_spoiler, dtype: int64

In [13]:
y_train.value_counts() / len(y_train)

False    0.93496
True     0.06504
Name: has_spoiler, dtype: float64

In [14]:
y_test.value_counts() / len(y_test)

False    0.934959
True     0.065041
Name: has_spoiler, dtype: float64

In [15]:
y_test.value_counts()

False    322101
True      22407
Name: has_spoiler, dtype: int64

In [16]:
len(X_test)

344508

# Handeling class imbalance
## sampling



In [11]:
#stats
def sampling_stats(Y_new):
    print("label dist" ,Y_new.value_counts() , "\n" )
    print("label ratio" ,Y_new.value_counts()/len(Y_new) )

In [12]:
#Random under sampling

from imblearn.under_sampling import RandomUnderSampler 
rus = RandomUnderSampler(random_state=42)
X_res, y_res = rus.fit_resample(X_train, y_train)
sampling_stats(y_res)


label dist False    67220
True     67220
Name: has_spoiler, dtype: int64 

label ratio False    0.5
True     0.5
Name: has_spoiler, dtype: float64


In [14]:
from imblearn.over_sampling import RandomOverSampler 
ros = RandomOverSampler(random_state=42)
X_ros, y_ros = ros.fit_resample(X_train, y_train)
sampling_stats(y_ros)

label dist False    966304
True     966304
Name: has_spoiler, dtype: int64 

label ratio False    0.5
True     0.5
Name: has_spoiler, dtype: float64


In [15]:
# Majority class random under sampling and minority class oversampling
from imblearn.pipeline import Pipeline 
over = RandomOverSampler(sampling_strategy=0.1)
under = RandomUnderSampler(sampling_strategy=0.5)
# define pipeline
pipeline = Pipeline(steps=[('o', over), ('u', under)])

In [16]:
X_ous, y_ous = pipeline.fit_resample(X_train, y_train)

In [17]:
sampling_stats(y_ous)

label dist False    193260
True      96630
Name: has_spoiler, dtype: int64 

label ratio False    0.666667
True     0.333333
Name: has_spoiler, dtype: float64


In [38]:
t = X_train['review_sentences_join'][0]

In [ ]:
def augment_text(df,samples=300,pr=0.2):
    aug_w2v.aug_p=pr
    new_text=[]
    
    ##selecting the minority class samples
    df_n=df[df.target==1].reset_index(drop=True)

    ## data augmentation loop
    for i in tqdm(np.random.randint(0,len(df_n),samples)):
        
            text = df_n.iloc[i]['text']
            augmented_text = aug_w2v.augment(text)
            new_text.append(augmented_text)
    
    
    ## dataframe
    new=pd.DataFrame({'text':new_text,'target':1})
    df=shuffle(df.append(new).reset_index(drop=True))
    return df

In [30]:
! pip install numpy requests nlpaug
! pip install nltk>=3.4.5
! pip install torch>=1.6.0 transformers>=4.11.3 sentencepiece


You should consider upgrading via the 'C:\Users\ayush\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


     -------------------------------------- 410.8/410.8 KB 2.3 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'C:\Users\ayush\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\ayush\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python39\\site-packages\\caffe2\\python\\serialized_test\\data\\operator_test\\piecewise_linear_transform_test.test_multi_predictions_params_from_arg.zip'

You should consider upgrading via the 'C:\Users\ayush\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [41]:
! pip install transformers>=4.11.3 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'C:\Users\ayush\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [36]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
  

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ayush\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ayush\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ayush\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\omw-1.4.zip.


True

In [34]:
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
import nlpaug.augmenter.word as naw
import nltk

from nlpaug.util import Action

C:\Users\ayush\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
import random
random.seed(42)

In [110]:
#data augumentation
from sklearn.utils import shuffle
aug_syn = naw.SynonymAug(aug_src='wordnet')
aug = naw.ContextualWordEmbsAug(
    model_path='bert-base-uncased', action="insert")

def augment_text(df,samples=60000):
    new_text=[]
    ##selecting the minority class samples
    df_n=df[df.has_spoiler==1].reset_index(drop=True)

    ## data augmentation loop
    for i in (np.random.randint(0,len(df_n),samples)):
            test_li = df_n.iloc[i]['review_sentences_join'].split('.')
            samp = min(len(test_li)-1 ,5 )
            for j in (np.random.randint(0,len(test_li),samp)):
                    text = test_li[j]
                    augmented_text = aug.augment(text)
                    test_li[j] = augmented_text
            new_text.append('.'.join(test_li))
            for k in (np.random.randint(0,len(test_li),samp)):
                    text = test_li[k]
                    augmented_text = aug_syn.augment(text)
                    test_li[k] = augmented_text
            new_text.append('.'.join(test_li))
    return new_text
    
  

In [111]:
X_train

,review_sentences,rating,genre,description,ratings_count,review_sentences_join,has_spoiler
1163910,"[[0, 'A brilliant continuation to Vampire Acad...",5,young-adult,"I wasn't free of my past, not yet.\nSydney's b...",142450,A brilliant continuation to Vampire Academy!I ...,False
213795,"[[0, '5 Stars'], [0, 'I was so Lucky to get an...",5,other,My War was over and I had lost. My captor remi...,2568,5 StarsI was so Lucky to get an ARC of this bo...,False
902682,"[[0, 'Really enjoyed this one although the end...",4,other,He's nobody's bitch. Until he gets a ride on t...,1402,Really enjoyed this one although the ending fe...,False
430509,"[[0, 'The author is a wonderful writer with he...",3,romance,Robert the Bruce consolidates lands and loyalt...,3458,The author is a wonderful writer with her othe...,True
1295031,"[[0, 'This book really upped the stakes and ch...",5,other,Amateur ghost-hunter Perry Palomino has battle...,5411,This book really upped the stakes and changed ...,False
...,...,...,...,...,...,...,...
914314,"[[0, 'More of a series of short stories than a...",3,fiction,Jennifer Egan's spellbinding interlocking narr...,122400,More of a series of short stories than an actu...,False
500185,"[[0, 'Okay, I admit it.....I bought this book ...",4,romance,The calm before the storm...\nI lived in the q...,3631,"Okay, I admit it.....I bought this book for th...",False
712447,"[[0, 'It was a very pleasant read.'], [0, 'Tru...",4,fantasy,"When Simon is kidnapped by the fey, he's amaze...",9084,"It was a very pleasant read.Truly.Unlike TMI, ...",True
482981,"[[0, ""I wasn't surprised that my favorite stor...",4,other,"January 27, 2027 - One day around the world.\n...",1146,I wasn't surprised that my favorite story in t...,False


In [112]:
data_x = X_train
data_x['has_spoiler'] = y_train

In [113]:
new_text = augment_text(data_x[['review_sentences_join' , 'has_spoiler']])


In [114]:
new=pd.DataFrame({'review_sentences_join':new_text,'has_spoiler':1})


In [119]:
df_aug=shuffle(data_x.append(new).reset_index(drop=True))

In [120]:
len(df_aug)

1153524

In [121]:
df_aug.has_spoiler.value_counts()/len(df_aug)

0    0.837697
1    0.162303
Name: has_spoiler, dtype: float64

In [122]:
df_aug.has_spoiler.value_counts(dropna = False)

0    966304
1    187220
Name: has_spoiler, dtype: int64

In [123]:
df_aug.to_csv('augumented_dataset_f.csv')

In [100]:
y_train.value_counts()

False    966304
True      67220
Name: has_spoiler, dtype: int64

In [ ]:
  ## dataframe
new=pd.DataFrame({'review_sentences_join':new_text,'target':1})
df=shuffle(df.append(new).reset_index(drop=True))
return df

In [101]:
df_aug

,review_sentences_join,has_spoiler
193373,"What I've heard of ""The Yellow Wallpaper""; Tha...",0
661953,"There really is a dying girl in this novel, so...",0
242315,T H O U G H T S:this is one of those books tha...,0
187643,It kept me flipping the pages.Very thought pro...,0
392904,Lots and lots and lots of sex.Then more sex.Bu...,0
...,...,...
959084,Great ending of the great story.From making am...,0
1032616,I can honestly admit that I'm a bit biased wit...,0
426251,"I don't know what the heck I just read, but it...",1
802904,I hate to say this as I was so looking forward...,0


In [51]:
aug = naw.SynonymAug(aug_src='wordnet')
# aug = naw.ContextualWordEmbsAug(
#     model_path='bert-base-uncased', action="insert")
augmented_text = aug.augment('This is what I love about good science fiction - it pushes your thinking about where things can go')
print("Original:")
print(t)
print("Augmented Text:")
print(augmented_text)

Original:
This is a special book.It started slow for about the first third, then in the middle third it started to get interesting, then the last third blew my mind.This is what I love about good science fiction - it pushes your thinking about where things can go.It is a 2015 Hugo winner, and translated from its original Chinese, which made it interesting in just a different way from most things I've read.For instance the intermixing of Chinese revolutionary history - how they kept accusing people of being "reactionaries", etc.It is a book about science, and aliens.The science described in the book is impressive - its a book grounded in physics and pretty accurate as far as I could tell.Though when it got to folding protons into 8 dimensions I think he was just making stuff up - interesting to think about though.But what would happen if our SETI stations received a message - if we found someone was out there - and the person monitoring and answering the signal on our side was disillusi

In [14]:
train_encodings = tokenizer(train_texts, truncation=True, padding='do_not_pad')


KeyboardInterrupt: 

In [13]:
train_encodings

1033524

In [1]:
train_encodings

NameError: name 'train_encodings' is not defined

In [1]:
# data sets
# data prep
!pip install zstandard


     -------------------------------------- 612.0/612.0 KB 1.8 MB/s eta 0:00:00


You should consider upgrading via the 'C:\Users\ayush\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [3]:
!pip install datasets

     -------------------------------------- 346.8/346.8 KB 1.5 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'C:\Users\ayush\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


     -------------------------------------- 140.6/140.6 KB 2.1 MB/s eta 0:00:00
     -------------------------------------- 554.9/554.9 KB 3.5 MB/s eta 0:00:00
     ---------------------------------------- 17.9/17.9 MB 2.3 MB/s eta 0:00:00
     -------------------------------------- 128.7/128.7 KB 7.4 MB/s eta 0:00:00
     ---------------------------------------- 60.6/60.6 KB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 122.1/122.1 KB ? eta 0:00:00


In [13]:
del df

In [3]:
# get Xtrain 
data_train = X_train
data_train['label'] = y_train
data_train[['review_sentences_join' , 'labels']].to_csv('train_sentences.csv' , index = False)

NameError: name 'X_train' is not defined

In [18]:
# get Xtest
data_train = X_test
data_train['label'] = y_test
data_train[['review_sentences_join' , 'labels']].to_csv('test_sentences.csv' , index = False)

In [19]:
del data_train

In [16]:
from datasets import load_dataset
datast_base = load_dataset("csv", data_files="train_sentences.csv")

Using custom data configuration default-35596291643b7c95


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 1003.42it/s]


Dataset csv downloaded and prepared to C:\Users\ayush\.cache\huggingface\datasets\csv\default-35596291643b7c95\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00,  7.42it/s]


In [32]:
datast_base_test = load_dataset("csv", data_files="test_sentences.csv")

Using custom data configuration default-95fae185a5d85c6e


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 1001.27it/s]


Dataset csv downloaded and prepared to C:\Users\ayush\.cache\huggingface\datasets\csv\default-95fae185a5d85c6e\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 27.48it/s]


DatasetDict({
    train: Dataset({
        features: ['review_sentences_join', 'label'],
        num_rows: 1033524
    })
})

In [21]:
# transformer check diffrent samples basline bert
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [24]:
def tokenize_function(examples):
    return tokenizer(examples["review_sentences_join"], padding="max_length", truncation=True)

In [25]:
tokenized_train = datast_base.map(tokenize_function, batched=True)

100%|██████████| 1034/1034 [08:24<00:00,  2.05ba/s]


In [33]:
tokenized_test = datast_base_test.map(tokenize_function, batched=True)

100%|██████████| 345/345 [02:40<00:00,  2.15ba/s]


In [34]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [28]:
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir="test_trainer")


In [29]:
from datasets import load_metric
metric = load_metric("accuracy")

In [30]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [31]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [41]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train['train'],
    eval_dataset=tokenized_test['train'],
    compute_metrics=compute_metrics,
)

In [39]:
tokenized_test

DatasetDict({
    train: Dataset({
        features: ['review_sentences_join', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 344508
    })
})

In [42]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: review_sentences_join. If review_sentences_join are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
C:\Users\ayush\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1033524
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 387573
  0%|          | 0/3 [02:15<?, ?it/s]


KeyboardInterrupt: 

In [43]:
# gpu and big data 
!pip install transformers datasets accelerate nvidia-ml-py3


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'C:\Users\ayush\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.



     -------------------------------------- 106.8/106.8 KB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using legacy 'setup.py install' for nvidia-ml-py3, since package 'wheel' is not installed.
  Running setup.py install for nvidia-ml-py3: started
  Running setup.py install for nvidia-ml-py3: finished with status 'done'


In [1]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [2]:
print_gpu_utilization()

NVMLError_LibraryNotFound: NVML Shared Library Not Found

In [3]:
!nvidia-smi

Sat May 21 19:26:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 462.62       Driver Version: 462.62       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 305... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   59C    P0    15W /  N/A |    106MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
import torch
torch.cuda.is_available()

C:\Users\ayush\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


False